In [108]:
# imports
import pandas as pd
import os
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
import category_encoders as ce

In [130]:
# loading data

# commenting this out while working locally in jupyter notebook
# FILE_PATH = os.path.join(os.path.dirname(__file__), 'data' 'initial_cleaned_data.csv')

FILE_PATH = '/home/zham/DS/model/data/initial_cleaned_data.csv'

df = pd.read_csv(FILE_PATH, index_col=None)

In [131]:
df.head()

,name,desc,goal,disable_communication,country,currency,deadline,launched_at,final_status,campaign_length,launch_year,launch_month,launch_day,launch_weekday
0,drawing for dollars,I like drawing pictures. and then i color them...,20.0,0,US,USD,2009-05-03 02:59:59,2009-04-24 15:52:03,1,8,2009,4,24,4
1,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,0,US,USD,2009-05-15 19:10:00,2009-04-28 23:26:32,0,17,2009,4,28,1
2,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,0,US,USD,2009-05-22 17:26:00,2009-05-12 17:39:58,0,10,2009,5,12,1
3,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,0,US,USD,2009-05-28 20:09:00,2009-04-28 20:58:50,1,30,2009,4,28,1
4,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,0,US,USD,2009-05-31 07:38:00,2009-05-01 08:22:21,0,30,2009,5,1,4


In [83]:
# majority classifier baseline
majority = train['final_status'].value_counts().index[0]

y_pred = len(test['final_status']) * [majority]
accuracy_score(test['final_status'], y_pred)

0.7319068428906417

In [132]:
encoder = ce.one_hot.OneHotEncoder(use_cat_names=True)

train = df[df['launch_year'] < 2014]
test = df[df['launch_year'] == 2014]

features = ['goal', 'campaign_length', 'launch_month', 'launch_day', 'launch_weekday', 'disable_communication',
'country', 'currency']
target = 'final_status'

X_train = train[features]
X_train_one_hot = encoder.fit_transform(X_train)

y_train = train[target]


X_test = test[features]
X_test_one_hot = encoder.transform(X_test)

y_test = test[target]


In [133]:
X_train_one_hot

,goal,campaign_length,launch_month,launch_day,launch_weekday,disable_communication,country_US,country_GB,country_CA,country_AU,country_NZ,currency_USD,currency_GBP,currency_CAD,currency_AUD,currency_NZD
0,20.0,8,4,24,4,0,1,0,0,0,0,1,0,0,0,0
1,300.0,17,4,28,1,0,1,0,0,0,0,1,0,0,0,0
2,30.0,10,5,12,1,0,1,0,0,0,0,1,0,0,0,0
3,500.0,30,4,28,1,0,1,0,0,0,0,1,0,0,0,0
4,2000.0,30,5,1,4,0,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54068,11000.0,60,12,30,0,0,1,0,0,0,0,1,0,0,0,0
54069,10000.0,60,12,30,0,0,0,0,0,0,1,0,0,0,0,1
54082,3000.0,60,12,30,0,0,1,0,0,0,0,1,0,0,0,0
54125,15000.0,60,12,31,1,0,1,0,0,0,0,1,0,0,0,0


In [134]:
# logistic regression baseline
# no matter what i did could not beat baseline
model = LogisticRegression(solver='lbfgs')

model.fit(X_train_one_hot, y_train)
model.score(X_test_one_hot, y_test)

0.7319068428906417